# Power Outage Durations

**Name(s)**: Aki Baidya and Dhruv Sehgal

**Website Link**: https://aki-baidya.github.io/power-outages/

In [233]:
import pandas as pd
import numpy as np
from pathlib import Path
from scipy.stats import chi2_contingency

import plotly.express as px
pd.options.plotting.backend = 'plotly'

# from dsc80_utils import * # Feel free to uncomment and use this.

## Step 1: Introduction

In [234]:
pd.set_option('display.max_columns', None) # to see all columns

# loading in the dataset
# we first changed the extension from .xlsx.xls to .xlsx
try:
    unclean_df = pd.read_excel('outage.xlsx', header=5) # omitting the rows above since they are not part of the data
    unclean_df = unclean_df.drop(0) # to remove the row that talks about the format of some data columns
    unclean_df = unclean_df.drop(columns=unclean_df.columns[:2]) # removing the 'variables' and 'OBS' columns

except FileNotFoundError:
    print('Error: \'outage.xlsx\' not found.')
display(unclean_df.head())

,YEAR,MONTH,U.S._STATE,POSTAL.CODE,NERC.REGION,CLIMATE.REGION,ANOMALY.LEVEL,CLIMATE.CATEGORY,OUTAGE.START.DATE,OUTAGE.START.TIME,OUTAGE.RESTORATION.DATE,OUTAGE.RESTORATION.TIME,CAUSE.CATEGORY,CAUSE.CATEGORY.DETAIL,HURRICANE.NAMES,OUTAGE.DURATION,DEMAND.LOSS.MW,CUSTOMERS.AFFECTED,RES.PRICE,COM.PRICE,IND.PRICE,TOTAL.PRICE,RES.SALES,COM.SALES,IND.SALES,TOTAL.SALES,RES.PERCEN,COM.PERCEN,IND.PERCEN,RES.CUSTOMERS,COM.CUSTOMERS,IND.CUSTOMERS,TOTAL.CUSTOMERS,RES.CUST.PCT,COM.CUST.PCT,IND.CUST.PCT,PC.REALGSP.STATE,PC.REALGSP.USA,PC.REALGSP.REL,PC.REALGSP.CHANGE,UTIL.REALGSP,TOTAL.REALGSP,UTIL.CONTRI,PI.UTIL.OFUSA,POPULATION,POPPCT_URBAN,POPPCT_UC,POPDEN_URBAN,POPDEN_UC,POPDEN_RURAL,AREAPCT_URBAN,AREAPCT_UC,PCT_LAND,PCT_WATER_TOT,PCT_WATER_INLAND
1,2011.0,7.0,Minnesota,MN,MRO,East North Central,-0.3,normal,2011-07-01 00:00:00,17:00:00,2011-07-03 00:00:00,20:00:00,severe weather,NaN,NaN,3060,NaN,70000.0,11.6,9.18,6.81,9.28,2332915,2114774,2113291,6562520,35.549073,32.225029,32.202431,2308736.0,276286.0,10673.0,2595696.0,88.944776,10.644005,0.411181,51268,47586,1.077376,1.6,4802,274182,1.751391,2.2,5348119.0,73.27,15.28,2279,1700.5,18.2,2.14,0.6,91.592666,8.407334,5.478743
2,2014.0,5.0,Minnesota,MN,MRO,East North Central,-0.1,normal,2014-05-11 00:00:00,18:38:00,2014-05-11 00:00:00,18:39:00,intentional attack,vandalism,NaN,1,NaN,NaN,12.12,9.71,6.49,9.28,1586986,1807756,1887927,5284231,30.032487,34.210389,35.727564,2345860.0,284978.0,9898.0,2640737.0,88.833534,10.791609,0.37482,53499,49091,1.089792,1.9,5226,291955,1.790002,2.2,5457125.0,73.27,15.28,2279,1700.5,18.2,2.14,0.6,91.592666,8.407334,5.478743
3,2010.0,10.0,Minnesota,MN,MRO,East North Central,-1.5,cold,2010-10-26 00:00:00,20:00:00,2010-10-28 00:00:00,22:00:00,severe weather,heavy wind,NaN,3000,NaN,70000.0,10.87,8.19,6.07,8.15,1467293,1801683,1951295,5222116,28.097672,34.501015,37.365983,2300291.0,276463.0,10150.0,2586905.0,88.920583,10.687018,0.392361,50447,47287,1.066826,2.7,4571,267895,1.706266,2.1,5310903.0,73.27,15.28,2279,1700.5,18.2,2.14,0.6,91.592666,8.407334,5.478743
4,2012.0,6.0,Minnesota,MN,MRO,East North Central,-0.1,normal,2012-06-19 00:00:00,04:30:00,2012-06-20 00:00:00,23:00:00,severe weather,thunderstorm,NaN,2550,NaN,68200.0,11.79,9.25,6.71,9.19,1851519,1941174,1993026,5787064,31.994099,33.54333,34.439329,2317336.0,278466.0,11010.0,2606813.0,88.895368,10.682239,0.422355,51598,48156,1.071476,0.6,5364,277627,1.932089,2.2,5380443.0,73.27,15.28,2279,1700.5,18.2,2.14,0.6,91.592666,8.407334,5.478743
5,2015.0,7.0,Minnesota,MN,MRO,East North Central,1.2,warm,2015-07-18 00:00:00,02:00:00,2015-07-19 00:00:00,07:00:00,severe weather,NaN,NaN,1740,250,250000.0,13.07,10.16,7.74,10.43,2028875,2161612,1777937,5970339,33.982576,36.20585,29.779498,2374674.0,289044.0,9812.0,2673531.0,88.821637,10.81132,0.367005,54431,49844,1.092027,1.7,4873,292023,1.668704,2.2,5489594.0,73.27,15.28,2279,1700.5,18.2,2.14,0.6,91.592666,8.407334,5.478743


## Step 2: Data Cleaning and Exploratory Data Analysis

In [235]:
df = unclean_df[['U.S._STATE', 'NERC.REGION', 'CLIMATE.REGION', 'OUTAGE.START.DATE', 'OUTAGE.START.TIME', 'OUTAGE.RESTORATION.DATE', 'OUTAGE.RESTORATION.TIME', 'CAUSE.CATEGORY', 'OUTAGE.DURATION', 'RES.PERCEN', 'COM.PERCEN', 'IND.PERCEN', 'PC.REALGSP.REL', 'POPPCT_URBAN']]
display(df.head())

,U.S._STATE,NERC.REGION,CLIMATE.REGION,OUTAGE.START.DATE,OUTAGE.START.TIME,OUTAGE.RESTORATION.DATE,OUTAGE.RESTORATION.TIME,CAUSE.CATEGORY,OUTAGE.DURATION,RES.PERCEN,COM.PERCEN,IND.PERCEN,PC.REALGSP.REL,POPPCT_URBAN
1,Minnesota,MRO,East North Central,2011-07-01 00:00:00,17:00:00,2011-07-03 00:00:00,20:00:00,severe weather,3060,35.549073,32.225029,32.202431,1.077376,73.27
2,Minnesota,MRO,East North Central,2014-05-11 00:00:00,18:38:00,2014-05-11 00:00:00,18:39:00,intentional attack,1,30.032487,34.210389,35.727564,1.089792,73.27
3,Minnesota,MRO,East North Central,2010-10-26 00:00:00,20:00:00,2010-10-28 00:00:00,22:00:00,severe weather,3000,28.097672,34.501015,37.365983,1.066826,73.27
4,Minnesota,MRO,East North Central,2012-06-19 00:00:00,04:30:00,2012-06-20 00:00:00,23:00:00,severe weather,2550,31.994099,33.54333,34.439329,1.071476,73.27
5,Minnesota,MRO,East North Central,2015-07-18 00:00:00,02:00:00,2015-07-19 00:00:00,07:00:00,severe weather,1740,33.982576,36.20585,29.779498,1.092027,73.27


In [236]:
df['OUTAGE.DURATION'] = df['OUTAGE.DURATION'].apply(lambda x: np.nan if x == 0 else x)
df = df[df['OUTAGE.DURATION'].notna()]

/var/folders/jk/ll2fwx_55klc6t5gz7x_24440000gn/T/ipykernel_63617/1239864593.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [237]:
unclean_df[unclean_df['OUTAGE.DURATION'].isna()].head(10)

,YEAR,MONTH,U.S._STATE,POSTAL.CODE,NERC.REGION,CLIMATE.REGION,ANOMALY.LEVEL,CLIMATE.CATEGORY,OUTAGE.START.DATE,OUTAGE.START.TIME,OUTAGE.RESTORATION.DATE,OUTAGE.RESTORATION.TIME,CAUSE.CATEGORY,CAUSE.CATEGORY.DETAIL,HURRICANE.NAMES,OUTAGE.DURATION,DEMAND.LOSS.MW,CUSTOMERS.AFFECTED,RES.PRICE,COM.PRICE,IND.PRICE,TOTAL.PRICE,RES.SALES,COM.SALES,IND.SALES,TOTAL.SALES,RES.PERCEN,COM.PERCEN,IND.PERCEN,RES.CUSTOMERS,COM.CUSTOMERS,IND.CUSTOMERS,TOTAL.CUSTOMERS,RES.CUST.PCT,COM.CUST.PCT,IND.CUST.PCT,PC.REALGSP.STATE,PC.REALGSP.USA,PC.REALGSP.REL,PC.REALGSP.CHANGE,UTIL.REALGSP,TOTAL.REALGSP,UTIL.CONTRI,PI.UTIL.OFUSA,POPULATION,POPPCT_URBAN,POPPCT_UC,POPDEN_URBAN,POPDEN_UC,POPDEN_RURAL,AREAPCT_URBAN,AREAPCT_UC,PCT_LAND,PCT_WATER_TOT,PCT_WATER_INLAND
23,2015.0,7.0,Tennessee,TN,SERC,Central,1.2,warm,2015-07-30 00:00:00,13:00:00,NaN,NaN,intentional attack,vandalism,NaN,NaN,0,0.0,10.31,10.27,7.31,9.7,4285049,3322562,1891937,9499547,45.10793,34.976005,19.916076,2783058.0,476405.0,1245.0,3260708.0,85.351341,14.610477,0.038182,42457,49844,0.851798,1.2,1692,282752,0.598404,0.4,6600299.0,66.39,12.02,1450.3,1076.2,55.6,7.05,1.72,97.843109,2.156891,2.156891
37,2016.0,7.0,Tennessee,TN,SERC,Central,-0.3,normal,2016-07-13 00:00:00,15:00:00,NaN,NaN,system operability disruption,public appeal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2812287.0,480032.0,1193.0,3293512.0,85.388698,14.57508,0.036223,43720,50660,0.863008,3,1813,290712,0.623641,0.5,6649404.0,66.39,12.02,1450.3,1076.2,55.6,7.05,1.72,97.843109,2.156891,2.156891
48,2016.0,4.0,Tennessee,TN,SERC,Central,1.1,warm,2016-04-27 00:00:00,13:36:00,NaN,NaN,intentional attack,vandalism,NaN,NaN,0,0.0,10.22,9.64,5.39,8.8,2517024,2577433,1681946,6776403,37.143954,38.035415,24.820631,2812287.0,480032.0,1193.0,3293512.0,85.388698,14.57508,0.036223,43720,50660,0.863008,3,1813,290712,0.623641,0.5,6649404.0,66.39,12.02,1450.3,1076.2,55.6,7.05,1.72,97.843109,2.156891,2.156891
50,2014.0,6.0,Wisconsin,WI,MRO,East North Central,0,normal,2014-06-27 00:00:00,13:21:00,NaN,NaN,fuel supply emergency,Coal,NaN,NaN,NaN,NaN,14.38,11.31,7.81,10.98,1706907,2012702,2037837,5757447,29.646942,34.958238,35.394803,2631430.0,345907.0,5465.0,2982802.0,88.22007,11.596713,0.183217,46676,49091,0.950806,1.9,4680,268742,1.741447,1.9,5759432.0,70.15,14.35,2123.3,1671.5,32.5,3.47,0.9,82.689019,17.312508,3.049041
183,2007.0,9.0,Texas,TX,WECC,South,-0.9,cold,2007-09-06 00:00:00,20:00:00,NaN,NaN,fuel supply emergency,NaN,NaN,NaN,NaN,NaN,12.44,9.88,7.76,10.3,13316662,10376502,9494213,33193019,40.118864,31.261097,28.603042,9166849.0,1413358.0,168586.0,10748834.0,85.282264,13.148942,1.568412,49068,49126,0.998819,2.5,27695,1169399,2.368311,10.9,23831983.0,84.7,9.35,2435.3,1539.9,15.2,3.35,0.58,97.258336,2.742036,2.090873
193,2014.0,4.0,Texas,TX,TRE,South,-0.2,normal,2014-04-03 00:00:00,00:00:00,NaN,NaN,fuel supply emergency,Coal,NaN,NaN,NaN,NaN,12.06,8.21,5.86,8.55,8199586,10173132,9461319,27846934,29.445202,36.532323,33.976161,10138874.0,1432478.0,101641.0,11672996.0,86.857513,12.271725,0.870736,52742,49091,1.074372,2,28121,1421759,1.977902,10.9,26979078.0,84.7,9.35,2435.3,1539.9,15.2,3.35,0.58,97.258336,2.742036,2.090873
233,2014.0,6.0,Texas,TX,TRE,South,0,normal,2014-06-06 00:00:00,13:00:00,NaN,NaN,fuel supply emergency,Coal,NaN,NaN,NaN,NaN,12.17,8.26,6.34,9.2,12861137,12571521,9260108,34707247,37.056056,36.221602,26.680618,10138874.0,1432478.0,101641.0,11672996.0,86.857513,12.271725,0.870736,52742,49091,1.074372,2,28121,1421759,1.977902,10.9,26979078.0,84.7,9.35,2435.3,1539.9,15.2,3.35,0.58,97.258336,2.742036,2.090873
240,2000.0,NaN,Texas,TX,FRCC,South,NaN,NaN,NaN,NaN,NaN,NaN,equipment failure,transformer outage,NaN,NaN,46,43000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8023266.0,1093414.0,61280.0,9299829.0,86.273264,11.757356,0.658937,45102,44745,1.007979,1.7,30908,944631,3.271965,10.3,20944499.0,84.7,9.35,2435.3,1539.9,15.2,3.35,0.58,97.258336,2.742036,2.090873
283,2015.0,12.0,Texas,TX,TRE,South,2.3,warm,2015-12-26 00:00:0

In [238]:
df['NERC.REGION'].isna().sum()
# there are no missing values in 'NERC.REGION'

np.int64(0)

In [239]:
df[df['CLIMATE.REGION'].isna()] # 5 rows with null values

,U.S._STATE,NERC.REGION,CLIMATE.REGION,OUTAGE.START.DATE,OUTAGE.START.TIME,OUTAGE.RESTORATION.DATE,OUTAGE.RESTORATION.TIME,CAUSE.CATEGORY,OUTAGE.DURATION,RES.PERCEN,COM.PERCEN,IND.PERCEN,PC.REALGSP.REL,POPPCT_URBAN
1516,Hawaii,HI,NaN,2008-12-26 00:00:00,18:13:00,2008-12-27 00:00:00,17:00:00,severe weather,1367.0,30.359526,32.964011,36.676462,1.04471,91.93
1517,Hawaii,PR,NaN,2011-05-02 00:00:00,17:06:00,2011-05-02 00:00:00,20:00:00,severe weather,174.0,29.256472,34.293692,36.449836,1.032026,91.93
1518,Hawaii,HECO,NaN,2006-10-15 00:00:00,07:09:00,2006-10-15 00:00:00,16:12:00,severe weather,543.0,29.754528,33.325929,36.919435,1.029626,91.93
1519,Hawaii,HECO,NaN,2006-06-01 00:00:00,14:12:00,2006-06-01 00:00:00,18:09:00,system operability disruption,237.0,29.775679,33.478432,36.745777,1.029626,91.93
1520,Hawaii,HECO,NaN,2006-10-15 00:00:00,07:09:00,2006-10-16 00:00:00,14:55:00,severe weather,1906.0,29.754528,33.325929,36.919435,1.029626,91.93


In [240]:
df[df['U.S._STATE'] == 'Hawaii']
# Hawaii does not have a 'CLIMATE.REGION'

,U.S._STATE,NERC.REGION,CLIMATE.REGION,OUTAGE.START.DATE,OUTAGE.START.TIME,OUTAGE.RESTORATION.DATE,OUTAGE.RESTORATION.TIME,CAUSE.CATEGORY,OUTAGE.DURATION,RES.PERCEN,COM.PERCEN,IND.PERCEN,PC.REALGSP.REL,POPPCT_URBAN
1516,Hawaii,HI,NaN,2008-12-26 00:00:00,18:13:00,2008-12-27 00:00:00,17:00:00,severe weather,1367.0,30.359526,32.964011,36.676462,1.04471,91.93
1517,Hawaii,PR,NaN,2011-05-02 00:00:00,17:06:00,2011-05-02 00:00:00,20:00:00,severe weather,174.0,29.256472,34.293692,36.449836,1.032026,91.93
1518,Hawaii,HECO,NaN,2006-10-15 00:00:00,07:09:00,2006-10-15 00:00:00,16:12:00,severe weather,543.0,29.754528,33.325929,36.919435,1.029626,91.93
1519,Hawaii,HECO,NaN,2006-06-01 00:00:00,14:12:00,2006-06-01 00:00:00,18:09:00,system operability disruption,237.0,29.775679,33.478432,36.745777,1.029626,91.93
1520,Hawaii,HECO,NaN,2006-10-15 00:00:00,07:09:00,2006-10-16 00:00:00,14:55:00,severe weather,1906.0,29.754528,33.325929,36.919435,1.029626,91.93


In [241]:
pd.set_option('display.max_rows', None)

In [242]:
df['CAUSE.CATEGORY'].isna().sum()
# all power outages have a valid cause

np.int64(0)

In [243]:
df[df['RES.PERCEN'].isna()]

,U.S._STATE,NERC.REGION,CLIMATE.REGION,OUTAGE.START.DATE,OUTAGE.START.TIME,OUTAGE.RESTORATION.DATE,OUTAGE.RESTORATION.TIME,CAUSE.CATEGORY,OUTAGE.DURATION,RES.PERCEN,COM.PERCEN,IND.PERCEN,PC.REALGSP.REL,POPPCT_URBAN
104,Michigan,RFC,East North Central,2016-07-08 00:00:00,19:00:00,2016-07-09 00:00:00,00:00:00,severe weather,300.0,NaN,NaN,NaN,0.85531,74.57
172,Texas,TRE,South,2016-07-05 00:00:00,02:45:00,2016-07-06 00:00:00,03:00:00,severe weather,1455.0,NaN,NaN,NaN,1.048243,84.7
219,Texas,TRE,South,2016-07-09 00:00:00,17:45:00,2016-07-11 00:00:00,14:00:00,severe weather,2655.0,NaN,NaN,NaN,1.048243,84.7
418,Washington,WECC,Northwest,2016-07-13 00:00:00,13:00:00,2016-07-13 00:00:00,13:01:00,intentional attack,1.0,NaN,NaN,NaN,1.140861,84.05
606,Pennsylvania,RFC,Northeast,2016-07-23 00:00:00,15:15:00,2016-07-23 00:00:00,19:53:00,system operability disruption,278.0,NaN,NaN,NaN,1.006277,78.66
744,North Carolina,SERC,Southeast,2016-07-08 00:00:00,20:50:00,2016-07-09 00:00:00,19:25:00,severe weather,1355.0,NaN,NaN,NaN,0.872365,66.09
763,North Carolina,SERC,Southeast,2016-07-07 00:00:00,05:53:00,2016-07-07 00:00:00,08:40:00,intentional attack,167.0,NaN,NaN,NaN,0.872365,66.09
828,Oregon,WECC,Northwest,2016-07-02 00:00:00,04:00:00,2016-07-04 00:00:00,00:40:00,intentional attack,2680.0,NaN,NaN,NaN,1.001796,81.03
901,Delaware,RFC,Northeast,2016-07-21 00:00:00,06:18:00,2016-07-21 00:00:00,14:45:00,intentional attack,507.0,NaN,NaN,NaN,1.254994,83.3
1420,Oklahoma,SPP,South,2016-07-14 00:00:00,14:44:00,2016-07-15 00:00:00,04:00:00,severe weather,796.0,NaN,NaN,NaN,0.876786,66.24


In [244]:
df[df['COM.PERCEN'].isna()].equals(df[df['RES.PERCEN'].isna()])

True

In [245]:
df[df['IND.PERCEN'].isna()].equals(df[df['RES.PERCEN'].isna()])
# NaN values for percentages are across the same rows

True

In [246]:
# imputing 'PERCEN' columns using the 
df['RES.PERCEN'] = df['RES.PERCEN'].fillna(df['RES.PERCEN'].mean())
df['COM.PERCEN'] = df['COM.PERCEN'].fillna(df['COM.PERCEN'].mean())
df['IND.PERCEN'] = df['IND.PERCEN'].fillna(df['IND.PERCEN'].mean())
display(df.head(15))

/var/folders/jk/ll2fwx_55klc6t5gz7x_24440000gn/T/ipykernel_63617/1602715756.py:2: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`

/var/folders/jk/ll2fwx_55klc6t5gz7x_24440000gn/T/ipykernel_63617/1602715756.py:3: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`

/var/folders/jk/ll2fwx_55klc6t5gz7x_24440000gn/T/ipykernel_63617/1602715756.py:4: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_opt

,U.S._STATE,NERC.REGION,CLIMATE.REGION,OUTAGE.START.DATE,OUTAGE.START.TIME,OUTAGE.RESTORATION.DATE,OUTAGE.RESTORATION.TIME,CAUSE.CATEGORY,OUTAGE.DURATION,RES.PERCEN,COM.PERCEN,IND.PERCEN,PC.REALGSP.REL,POPPCT_URBAN
1,Minnesota,MRO,East North Central,2011-07-01 00:00:00,17:00:00,2011-07-03 00:00:00,20:00:00,severe weather,3060.0,35.549073,32.225029,32.202431,1.077376,73.27
2,Minnesota,MRO,East North Central,2014-05-11 00:00:00,18:38:00,2014-05-11 00:00:00,18:39:00,intentional attack,1.0,30.032487,34.210389,35.727564,1.089792,73.27
3,Minnesota,MRO,East North Central,2010-10-26 00:00:00,20:00:00,2010-10-28 00:00:00,22:00:00,severe weather,3000.0,28.097672,34.501015,37.365983,1.066826,73.27
4,Minnesota,MRO,East North Central,2012-06-19 00:00:00,04:30:00,2012-06-20 00:00:00,23:00:00,severe weather,2550.0,31.994099,33.543330,34.439329,1.071476,73.27
5,Minnesota,MRO,East North Central,2015-07-18 00:00:00,02:00:00,2015-07-19 00:00:00,07:00:00,severe weather,1740.0,33.982576,36.205850,29.779498,1.092027,73.27
6,Minnesota,MRO,East North Central,2010-11-13 00:00:00,15:00:00,2010-11-14 00:00:00,22:00:00,severe weather,1860.0,31.192784,33.235842,35.538160,1.066826,73.27
7,Minnesota,MRO,East North Central,2010-07-17 00:00:00,20:30:00,2010-07-19 00:00:00,22:00:00,severe weather,2970.0,34.314656,32.954501,32.701651,1.066826,73.27
8,Minnesota,MRO,East North Central,2005-06-08 00:00:00,04:00:00,2005-06-10 00:00:00,22:00:00,severe weather,3960.0,33.586173,34.515286,31.861839,1.090559,73.27
9,Minnesota,MRO,East North Central,2015-03-16 00:00:00,07:31:00,2015-03-16 00:00:00,10:06:00,intentional attack,155.0,32.936916,34.947243,32.071962,1.092027,73.27
10,Minnesota,MRO,East North Central,2013-06-21 00:00:00,17:39:00,2013-06-24 00:00:00,06:00:00,severe weather,3621.0,30.754553,34.515140,34.702296,1.084366,73.27


In [247]:
df['PC.REALGSP.REL'].isna().sum()
# no null values

np.int64(0)

In [248]:
df['POPPCT_URBAN'].isna().sum()
# no null values again

np.int64(0)

In [249]:
# converting to pd.Timestamp
outage_start_date_str = pd.to_datetime(df['OUTAGE.START.DATE']).astype(str)
outage_start_time_str = pd.to_datetime(df['OUTAGE.START.TIME'], format='%H:%M:%S').dt.time.astype(str)
outage_start = pd.to_datetime(outage_start_date_str + ' ' + outage_start_time_str)

outage_restoration_date_str = pd.to_datetime(df['OUTAGE.RESTORATION.DATE']).astype(str)
outage_restoration_time_str = pd.to_datetime(df['OUTAGE.RESTORATION.TIME'], format='%H:%M:%S').dt.time.astype(str)
outage_restoration = pd.to_datetime(outage_restoration_date_str + ' ' + outage_restoration_time_str)

df['OUTAGE.START'] = outage_start
df['OUTAGE.RESTORATION'] = outage_restoration
df = df.drop(columns=['OUTAGE.START.DATE', 'OUTAGE.START.TIME', 'OUTAGE.RESTORATION.DATE', 'OUTAGE.RESTORATION.TIME'])

In [250]:
display(df.head()) # head of the cleaned dataframe

,U.S._STATE,NERC.REGION,CLIMATE.REGION,CAUSE.CATEGORY,OUTAGE.DURATION,RES.PERCEN,COM.PERCEN,IND.PERCEN,PC.REALGSP.REL,POPPCT_URBAN,OUTAGE.START,OUTAGE.RESTORATION
1,Minnesota,MRO,East North Central,severe weather,3060.0,35.549073,32.225029,32.202431,1.077376,73.27,2011-07-01 17:00:00,2011-07-03 20:00:00
2,Minnesota,MRO,East North Central,intentional attack,1.0,30.032487,34.210389,35.727564,1.089792,73.27,2014-05-11 18:38:00,2014-05-11 18:39:00
3,Minnesota,MRO,East North Central,severe weather,3000.0,28.097672,34.501015,37.365983,1.066826,73.27,2010-10-26 20:00:00,2010-10-28 22:00:00
4,Minnesota,MRO,East North Central,severe weather,2550.0,31.994099,33.543330,34.439329,1.071476,73.27,2012-06-19 04:30:00,2012-06-20 23:00:00
5,Minnesota,MRO,East North Central,severe weather,1740.0,33.982576,36.205850,29.779498,1.092027,73.27,2015-07-18 02:00:00,2015-07-19 07:00:00


### Univariate Analysis

In [251]:
cause_counts = pd.DataFrame(df['CAUSE.CATEGORY'].value_counts().reset_index())
pie_fig = px.pie(cause_counts, values='count', names='CAUSE.CATEGORY', title='Relative Frequencies of Causes of Power Outages')
pie_fig.show()
# This plot shows that the most common cause of power outages is severe weather, and the least common is fuel supply emergency

# -- Exporting plotly figure as HTML --
pie_fig.write_html('assets/pie-chart.html', include_plotlyjs='cdn')


In [252]:
hist_fig = px.histogram(df, x='U.S._STATE', title='Number of Power Outages By State', labels={'U.S._STATE': 'State'})
hist_fig.update_layout(yaxis_title="Number of Power Outages")
hist_fig.update_xaxes(tickangle=-45)
hist_fig.show()

hist_fig.write_html('assets/num_outages.html', include_plotlyjs='cdn')

### Bivariate Analysis

In [253]:
avg_dur_fig = px.bar(df.groupby('CLIMATE.REGION')['OUTAGE.DURATION'].mean().sort_values(ascending=False), y='OUTAGE.DURATION', title='Mean Outage Duration By Climate Region')
avg_dur_fig.update_layout(xaxis_title="Climate Region", yaxis_title="Mean Outage Duration (Minutes)")
avg_dur_fig.update_xaxes(tickangle=-45)
avg_dur_fig.show()

# this bar plot shows that the 'CLIMATE.REGION' in which power takes the longest to restore on average is 'East North Central'. The minimum is at 'West North Central'.

avg_dur_fig.write_html('assets/avg_dur_fig.html', include_plotlyjs='cdn')

In [254]:
outages_per_region = px.bar(df.groupby('CLIMATE.REGION')['OUTAGE.DURATION'].count().sort_values(ascending=False), y='OUTAGE.DURATION', title='Number of Power Outages By Climate Region')
outages_per_region.update_layout(xaxis_title="Climate Region", yaxis_title="Number of Outages")
outages_per_region.update_xaxes(tickangle=-45)
outages_per_region.show()
# this bar plot shows that the 'CLIMATE.REGION' in which power outages happen the most frequently is different, becoming 'Northeast' this time. Both have the minimum at 'West North Central'.

avg_dur_fig.write_html('assets/outages_per_region_fig.html', include_plotlyjs='cdn')

### Interesting Aggregates

In [255]:
# Funciton to calculate the average time of day of a pd.Timestamp series, without including the date
def avg_time(series):
    mean_seconds = (series.dt.hour * 3600 + series.dt.minute * 60 + series.dt.second).mean()
    return pd.to_datetime(mean_seconds, unit='s').round(freq='s').time()

In [256]:
df.groupby('U.S._STATE')['OUTAGE.START'].agg(avg_time)
pd.pivot_table(df, values='OUTAGE.START', index='CAUSE.CATEGORY', columns=['CLIMATE.REGION'], aggfunc=avg_time)

CLIMATE.REGION,Central,East North Central,Northeast,Northwest,South,Southeast,Southwest,West,West North Central
CAUSE.CATEGORY,,,,,,,,,
equipment failure,13:00:48,08:35:00,11:15:45,16:47:00,10:46:00,16:50:15,15:39:36,12:00:20,09:46:00
fuel supply emergency,08:36:15,07:50:15,12:04:09,01:48:00,09:52:30,NaN,09:14:00,13:17:18,NaN
intentional attack,11:23:35,12:19:22,10:26:53,12:21:13,10:55:33,12:05:40,11:15:06,10:32:22,12:51:00
islanding,18:19:00,12:01:00,22:44:00,16:03:20,18:13:30,NaN,09:22:00,12:26:43,14:50:12
public appeal,14:00:00,14:07:00,13:00:00,14:02:00,13:36:16,15:00:00,10:00:00,14:08:07,08:10:30
severe weather,13:11:27,13:32:09,13:30:53,09:33:53,12:26:11,13:36:49,09:15:06,11:46:49,17:20:00
system operability disruption,15:16:30,11:54:20,15:21:39,15:28:00,13:27:18,12:11:44,14:02:45,12:54:09,NaN


## Step 3: Assessment of Missingness

In [257]:
df.head()

,U.S._STATE,NERC.REGION,CLIMATE.REGION,CAUSE.CATEGORY,OUTAGE.DURATION,RES.PERCEN,COM.PERCEN,IND.PERCEN,PC.REALGSP.REL,POPPCT_URBAN,OUTAGE.START,OUTAGE.RESTORATION
1,Minnesota,MRO,East North Central,severe weather,3060.0,35.549073,32.225029,32.202431,1.077376,73.27,2011-07-01 17:00:00,2011-07-03 20:00:00
2,Minnesota,MRO,East North Central,intentional attack,1.0,30.032487,34.210389,35.727564,1.089792,73.27,2014-05-11 18:38:00,2014-05-11 18:39:00
3,Minnesota,MRO,East North Central,severe weather,3000.0,28.097672,34.501015,37.365983,1.066826,73.27,2010-10-26 20:00:00,2010-10-28 22:00:00
4,Minnesota,MRO,East North Central,severe weather,2550.0,31.994099,33.543330,34.439329,1.071476,73.27,2012-06-19 04:30:00,2012-06-20 23:00:00
5,Minnesota,MRO,East North Central,severe weather,1740.0,33.982576,36.205850,29.779498,1.092027,73.27,2015-07-18 02:00:00,2015-07-19 07:00:00


In [258]:
df.isnull().any()

U.S._STATE            False
NERC.REGION           False
CLIMATE.REGION         True
CAUSE.CATEGORY        False
OUTAGE.DURATION       False
RES.PERCEN            False
COM.PERCEN            False
IND.PERCEN            False
PC.REALGSP.REL        False
POPPCT_URBAN          False
OUTAGE.START          False
OUTAGE.RESTORATION    False
dtype: bool

In [259]:
df[df['CLIMATE.REGION'].isna()]

,U.S._STATE,NERC.REGION,CLIMATE.REGION,CAUSE.CATEGORY,OUTAGE.DURATION,RES.PERCEN,COM.PERCEN,IND.PERCEN,PC.REALGSP.REL,POPPCT_URBAN,OUTAGE.START,OUTAGE.RESTORATION
1516,Hawaii,HI,NaN,severe weather,1367.0,30.359526,32.964011,36.676462,1.04471,91.93,2008-12-26 18:13:00,2008-12-27 17:00:00
1517,Hawaii,PR,NaN,severe weather,174.0,29.256472,34.293692,36.449836,1.032026,91.93,2011-05-02 17:06:00,2011-05-02 20:00:00
1518,Hawaii,HECO,NaN,severe weather,543.0,29.754528,33.325929,36.919435,1.029626,91.93,2006-10-15 07:09:00,2006-10-15 16:12:00
1519,Hawaii,HECO,NaN,system operability disruption,237.0,29.775679,33.478432,36.745777,1.029626,91.93,2006-06-01 14:12:00,2006-06-01 18:09:00
1520,Hawaii,HECO,NaN,severe weather,1906.0,29.754528,33.325929,36.919435,1.029626,91.93,2006-10-15 07:09:00,2006-10-16 14:55:00


In our cleaned dataset, the only column with missing values is `'CLIMATE.REGION'`. Since the only rows for which values are missing are for `'U.S._STATE' == Hawaii`, this missingness depends on the `'U.S._STATE'` column, making it MAR. Before imputing missing values in `'RES.PERCEN'`, `'COM.PERCEN'`, and `'IND.PERCEN'`, the missing values did not seem to depend on themselves, and hence we cannot be sure if the missingness mechanism is NMAR.

In [260]:
# doesn't matter on GSP
observed = np.abs(df[df['CLIMATE.REGION'].notna()]['PC.REALGSP.REL'].mean() - df[df['CLIMATE.REGION'].isna()]['PC.REALGSP.REL'].mean())

sim_df = df[['CLIMATE.REGION', 'PC.REALGSP.REL']]
sims = []
for _ in range(1000):
    sim_df = sim_df.assign(CLIMATE_REGION_SHUFFLED = np.random.permutation(sim_df['CLIMATE.REGION']))
    sim_stat = np.abs(sim_df[sim_df['CLIMATE_REGION_SHUFFLED'].notna()]['PC.REALGSP.REL'].mean() - sim_df[sim_df['CLIMATE_REGION_SHUFFLED'].isna()]['PC.REALGSP.REL'].mean())
    sims.append(sim_stat)

p_value = np.mean(sims >= observed)
p_value

np.float64(0.9)

In [261]:
sim_df = sim_df.assign(CLIMATE_REGION_MISSINGNESS = sim_df['CLIMATE.REGION'].isna())
cli_mi = px.bar(sim_df.groupby('CLIMATE_REGION_MISSINGNESS')['PC.REALGSP.REL'].mean().reset_index(), orientation='h', y='CLIMATE_REGION_MISSINGNESS', x='PC.REALGSP.REL')
cli_mi.show()

cli_mi.write_html('assets/cli_mi.html', include_plotlyjs='cdn')


In [262]:
# p_value comes out super small, hence these columns are related on missingness

chi_df = df[['NERC.REGION', 'CLIMATE.REGION']].assign(MISSING_CLIMATE_REGION = df['CLIMATE.REGION'].isnull().astype(int))
observed_table = pd.crosstab(chi_df['MISSING_CLIMATE_REGION'], chi_df['NERC.REGION'])
observed_stat = chi2_contingency(observed_table)[0]
observed_stat

chi_stats = []
for _ in range(1000):
    shuffled = np.random.permutation(chi_df['MISSING_CLIMATE_REGION'])
    chi_table = pd.crosstab(shuffled, chi_df['NERC.REGION'])
    chi_stat = chi2_contingency(chi_table)[0]
    chi_stats.append(chi_stat)

p_value = np.mean(chi_stats >= observed_stat)
p_value

np.float64(0.0)

In [263]:
chi_viz = pd.DataFrame({'Simulated Chi': chi_stats})
fig = px.histogram(chi_viz)
fig.add_vline(x=observed_stat, line_width=3, line_dash='dash', line_color='green', annotation_text='Observed Test Statistic', annotation_position='top left')
fig.show()

fig.write_html('assets/sim_chi.html', include_plotlyjs='cdn')

## Step 4: Hypothesis Testing

**Null Hypothesis**: Outage Durations are equal in length regardless of whether the majority of electricity consumption in the state is residential or industrial
**Alternate Hypothesis**: Outage Durations are longer when the majority of electricity consumption in the state is residential, as compared to industrial

I define the majority to be more than 50%. Hence, we would be comparing `'OUTAGE.DURATION'` across rows with `'RES.PERCEN' > 50.0` and `'IND.PERCEN' > 50.0`.

Our text statistic would be subtracting the mean `'OUTAGE.DURATION'` of industrially dominated states from the mean `'OUTAGE.DURATION'` of residentially dominated states. This would be a permutation test.

Since we only have ~1500 rows, we chose a p-value of 0.05.

In [264]:
residential = df[df['RES.PERCEN'] > 50.0]['OUTAGE.DURATION']
industrial = df[df['IND.PERCEN'] > 50.0]['OUTAGE.DURATION']
observed_test_stat = residential.mean() - industrial.mean()

# calculating simulated statistics,
perm_df = df[['RES.PERCEN', 'IND.PERCEN', 'OUTAGE.DURATION']]
perm_stats = []
for _ in range(1000):
    shuffled = np.random.permutation(df['OUTAGE.DURATION'])
    perm_df = perm_df.assign(SHUFFLED_OUTAGE_DURATION = shuffled)
    residential_sim = perm_df[perm_df['RES.PERCEN'] > 50.0]['SHUFFLED_OUTAGE_DURATION']
    industrial_sim = perm_df[perm_df['IND.PERCEN'] > 50.0]['SHUFFLED_OUTAGE_DURATION']
    perm_stat = residential_sim.mean() - industrial_sim.mean()
    perm_stats.append(perm_stat)

p_value = np.mean(perm_stats >= observed_test_stat)
p_value # we reject the null, < 0.05

np.float64(0.029)

In [265]:
perm_viz = pd.DataFrame({'Test Statistic': perm_stats})
fig2 = px.histogram(perm_viz)
fig2.add_vline(x=observed_test_stat, line_width=3, line_dash='dash', line_color='red', annotation_text='Observed Test Statistic')
fig2.show()

fig2.write_html('assets/hypo_test.html', include_plotlyjs='cdn')

## Step 5: Framing a Prediction Problem

Our prediction model attempts to predict the duration of a power outage. Since power outage duration is an example of a quantitative, continuous variable, our problem will involve linear regression. 


In [266]:
df.head(10)

,U.S._STATE,NERC.REGION,CLIMATE.REGION,CAUSE.CATEGORY,OUTAGE.DURATION,RES.PERCEN,COM.PERCEN,IND.PERCEN,PC.REALGSP.REL,POPPCT_URBAN,OUTAGE.START,OUTAGE.RESTORATION
1,Minnesota,MRO,East North Central,severe weather,3060.0,35.549073,32.225029,32.202431,1.077376,73.27,2011-07-01 17:00:00,2011-07-03 20:00:00
2,Minnesota,MRO,East North Central,intentional attack,1.0,30.032487,34.210389,35.727564,1.089792,73.27,2014-05-11 18:38:00,2014-05-11 18:39:00
3,Minnesota,MRO,East North Central,severe weather,3000.0,28.097672,34.501015,37.365983,1.066826,73.27,2010-10-26 20:00:00,2010-10-28 22:00:00
4,Minnesota,MRO,East North Central,severe weather,2550.0,31.994099,33.543330,34.439329,1.071476,73.27,2012-06-19 04:30:00,2012-06-20 23:00:00
5,Minnesota,MRO,East North Central,severe weather,1740.0,33.982576,36.205850,29.779498,1.092027,73.27,2015-07-18 02:00:00,2015-07-19 07:00:00
6,Minnesota,MRO,East North Central,severe weather,1860.0,31.192784,33.235842,35.538160,1.066826,73.27,2010-11-13 15:00:00,2010-11-14 22:00:00
7,Minnesota,MRO,East North Central,severe weather,2970.0,34.314656,32.954501,32.701651,1.066826,73.27,2010-07-17 20:30:00,2010-07-19 22:00:00
8,Minnesota,MRO,East North Central,severe weather,3960.0,33.586173,34.515286,31.861839,1.090559,73.27,2005-06-08 04:00:00,2005-06-10 22:00:00
9,Minnesota,MRO,East North Central,intentional attack,155.0,32.936916,34.947243,32.071962,1.092027,73.27,2015-03-16 07:31:00,2015-03-16 10:06:00
10,Minnesota,MRO,East North Central,severe weather,3621.0,30.754553,34.515140,34.702296,1.084366,73.27,2013-06-21 17:39:00,2013-06-24 06:00:00


## Step 6: Baseline Model

In [267]:
# TODO

## Step 7: Final Model

In [268]:
# TODO

## Step 8: Fairness Analysis

In [269]:
# TODO